In [62]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
import pickle

import streamlit as st
import pickle
import numpy as np

In [63]:
df = pd.read_csv("datasets_13996_18858_WA_Fn-UseC_-Telco-Customer-Churn - datasets_13996_18858_WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(6)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes


In [64]:
df.shape

(7043, 21)

In [65]:
df.isnull().sum(axis = 0)

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [66]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [97]:
# Check for values present in each column
# df.nunique(axis = 0)
df.agg(['count', 'size', 'nunique']).T # nunique is what we are looking for

,count,size,nunique
customerID,7043,7043,7043
gender,7043,7043,2
SeniorCitizen,7043,7043,2
Partner,7043,7043,2
Dependents,7043,7043,2
tenure,7043,7043,73
PhoneService,7043,7043,2
MultipleLines,7043,7043,3
InternetService,7043,7043,3
OnlineSecurity,7043,7043,3


In [68]:
# Let's remove the customerID column since the ID doesn't affect our model
df_v2 = df.drop(["customerID"], axis=1)
df_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


In [69]:
# Machine learning algorithms work best with numerical data. However, in our dataset, 
# we have only 4 numerical columns: SeniorCitizen, tenure, MonthlyCharges and TotalCharges. 
# These remaining columns contain data in textual format; we need to convert them to numeric columns.

# df_v2

lb_e = LabelEncoder()
for col in df_v2.columns:
    if (df_v2[col].dtype == 'object'):
        df_v2[col] = lb_e.fit_transform(df_v2[col])
df_v2

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,24,1,2,0,2,0,2,2,2,2,1,1,3,84.80,1990.50,0
7039,0,0,1,1,72,1,2,1,0,2,2,0,2,2,1,1,1,103.20,7362.90,0
7040,0,0,1,1,11,0,1,0,2,0,0,0,0,0,0,1,2,29.60,346.45,0
7041,1,1,1,0,4,1,2,1,0,0,0,0,0,0,0,1,3,74.40,306.60,1


In [75]:
# df_v2["tenure"].sort_values()
newdata = df_v2.corr()
newdata

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
gender,1.000000,-0.001874,-0.001808,0.010517,0.005106,-0.006488,-0.006739,-0.000863,-0.015017,-0.012057,0.000549,-0.006825,-0.006421,-0.008743,0.000126,-0.011754,0.017352,-0.014569,0.000048,-0.008612
SeniorCitizen,-0.001874,1.000000,0.016479,-0.211185,0.016567,0.008576,0.146185,-0.032310,-0.128221,-0.013632,-0.021398,-0.151268,0.030776,0.047266,-0.142554,0.156530,-0.038551,0.220173,0.102411,0.150889
Partner,-0.001808,0.016479,1.000000,0.452676,0.379697,0.017706,0.142410,0.000891,0.150828,0.153130,0.166330,0.126733,0.137341,0.129574,0.294806,-0.014877,-0.154798,0.096848,0.319072,-0.150448
Dependents,0.010517,-0.211185,0.452676,1.000000,0.159712,-0.001762,-0.024991,0.044590,0.152166,0.091015,0.080537,0.133524,0.046885,0.021321,0.243187,-0.111377,-0.040292,-0.113890,0.064653,-0.164221
tenure,0.005106,0.016567,0.379697,0.159712,1.000000,0.008448,0.343032,-0.030359,0.325468,0.370876,0.371105,0.322942,0.289373,0.296866,0.671607,0.006152,-0.370436,0.247900,0.825880,-0.352229
PhoneService,-0.006488,0.008576,0.017706,-0.001762,0.008448,1.000000,-0.020538,0.387436,-0.015198,0.024105,0.003727,-0.019158,0.055353,0.043870,0.002247,0.016505,-0.004184,0.247398,0.113008,0.011942
MultipleLines,-0.006739,0.146185,0.142410,-0.024991,0.343032,-0.020538,1.000000,-0.109216,0.007141,0.117327,0.122318,0.011466,0.175059,0.180957,0.110842,0.165146,-0.176793,0.433576,0.453202,0.038037
InternetService,-0.000863,-0.032310,0.000891,0.044590,-0.030359,0.387436,-0.109216,1.000000,-0.028416,0.036138,0.044944,-0.026047,0.107417,0.098350,0.099721,-0.138625,0.086140,-0.323260,-0.175691,-0.047291
OnlineSecurity,-0.015017,-0.128221,0.150828,0.152166,0.325468,-0.015198,0.007141,-0.028416,1.000000,0.185126,0.175985,0.285028,0.044669,0.055954,0.374416,-0.157641,-0.096726,-0.053878,0.254473,-0.289309
OnlineBackup,-0.012057,-0.013632,0.153130,0.091015,0.370876,0.024105,0.117327,0.036138,0.185126,1.000000,0.187757,0.195748,0.147186,0.136722,0.280980,-0.013370,-0.124847,0.119777,0.375556,-0.195525


In [96]:
df_v2.columns.values.tolist()

['SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [95]:
# Now we need to be practical and choose columns that are approriate for our model
df_v2 = df_v2.drop(['gender', 'InternetService', 'StreamingMovies', 'StreamingTV', 'PhoneService', 'MultipleLines'], axis=1)

KeyError: "['gender' 'InternetService' 'StreamingMovies' 'StreamingTV' 'PhoneService'\n 'MultipleLines'] not found in axis"

In [76]:
df_v2.columns[df_v2.isna().any()].tolist()

Index(['TotalCharges'], dtype='object')

In [88]:
df_v2["TotalCharges"].value_counts()

count      7032
size       7043
nunique    6530
Name: TotalCharges, dtype: int64

In [58]:
lb= LabelEncoder()

In [71]:
newdata=df

In [72]:
le= LabelEncoder()
newdata['gender']=le.fit_transform(newdata['gender'])
newdata['Dependents']=le.fit_transform(newdata['Dependents'])
newdata['PhoneService']=le.fit_transform(newdata['PhoneService'])
newdata['InternetService']=le.fit_transform(newdata['InternetService'])
newdata['OnlineSecurity']=le.fit_transform(newdata['OnlineSecurity'])
newdata['OnlineBackup']=le.fit_transform(newdata['OnlineBackup'])
newdata['TechSupport']=le.fit_transform(newdata['TechSupport'])
newdata['PaymentMethod']=le.fit_transform(newdata['PaymentMethod'])
newdata['PaperlessBilling']=le.fit_transform(newdata['PaperlessBilling'])
newdata['Contract']=le.fit_transform(newdata['Contract'])
newdata['Churn']=le.fit_transform(newdata['Churn'])

In [73]:
newdata

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,Yes,0,1,0,No phone service,0,0,...,No,0,No,No,0,1,2,29.85,29.85,0
1,5575-GNVDE,1,0,No,0,34,1,No,0,2,...,Yes,0,No,No,1,0,3,56.95,1889.5,0
2,3668-QPYBK,1,0,No,0,2,1,No,0,2,...,No,0,No,No,0,1,3,53.85,108.15,1
3,7795-CFOCW,1,0,No,0,45,0,No phone service,0,2,...,Yes,2,No,No,1,0,0,42.30,1840.75,0
4,9237-HQITU,0,0,No,0,2,1,No,1,0,...,No,0,No,No,0,1,2,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,1,0,Yes,1,24,1,Yes,0,2,...,Yes,2,Yes,Yes,1,1,3,84.80,1990.5,0
7039,2234-XADUH,0,0,Yes,1,72,1,Yes,1,0,...,Yes,0,Yes,Yes,1,1,1,103.20,7362.9,0
7040,4801-JZAZL,0,0,Yes,1,11,0,No phone service,0,2,...,No,0,No,No,0,1,2,29.60,346.45,0
7041,8361-LTMKD,1,1,Yes,0,4,1,Yes,1,0,...,No,0,No,No,0,1,3,74.40,306.6,1


In [99]:
newdata.corr()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
gender,1.000000,-0.061591,-0.148268,-0.058452,-0.166190,-0.104485,-0.158257,-0.037872,-0.113876,-0.164926,-0.160583,-0.120330,-0.201666,-0.202232,-0.130920,-0.087782,0.077879,-0.168625,-0.204315,0.011678
SeniorCitizen,-0.061591,1.000000,-0.222421,-0.535441,-0.186818,-0.024462,0.249038,-0.214097,-0.434644,-0.212288,-0.231855,-0.463216,-0.053998,-0.027148,-0.441435,0.394847,-0.044458,0.376047,-0.003975,0.397328
Partner,-0.148268,-0.222421,1.000000,0.682231,0.591573,-0.175257,0.183646,-0.150684,0.372134,0.331136,0.338831,0.334255,0.181590,0.172002,0.553907,-0.232268,-0.474267,0.016164,0.458025,-0.506617
Dependents,-0.058452,-0.535441,0.682231,1.000000,0.277058,-0.148227,-0.179148,0.060120,0.373573,0.187421,0.164001,0.329121,-0.032705,-0.064867,0.453807,-0.422119,-0.161083,-0.368484,0.055669,-0.475145
tenure,-0.166190,-0.186818,0.591573,0.277058,1.000000,-0.177641,0.476522,-0.253653,0.586040,0.642559,0.662654,0.609289,0.472508,0.486118,0.860940,-0.171361,-0.755709,0.291769,0.908650,-0.712968
PhoneService,-0.104485,-0.024462,-0.175257,-0.148227,-0.177641,1.000000,-0.160749,0.502726,-0.189820,-0.128999,-0.167864,-0.203963,-0.050156,-0.067506,-0.176702,-0.028010,0.021174,0.109590,-0.101762,0.066562
MultipleLines,-0.158257,0.249038,0.183646,-0.179148,0.476522,-0.160749,1.000000,-0.467986,-0.030079,0.201725,0.216712,-0.006905,0.328175,0.342926,0.130606,0.319825,-0.513590,0.686866,0.655802,-0.024575
InternetService,-0.037872,-0.214097,-0.150684,0.060120,-0.253653,0.502726,-0.467986,1.000000,-0.087089,-0.112746,-0.104919,-0.095866,-0.084312,-0.099208,0.010656,-0.401317,0.238207,-0.603535,-0.463123,-0.103790
OnlineSecurity,-0.113876,-0.434644,0.372134,0.373573,0.586040,-0.189820,-0.030079,-0.087089,1.000000,0.438150,0.419782,0.625168,0.080393,0.098090,0.708377,-0.472877,-0.346467,-0.208523,0.393358,-0.706388
OnlineBackup,-0.164926,-0.212288,0.331136,0.187421,0.642559,-0.128999,0.201725,-0.112746,0.438150,1.000000,0.425823,0.454123,0.260191,0.253949,0.574436,-0.204809,-0.461623,0.102239,0.572371,-0.565740


In [100]:
newdata

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
gender,1.000000,-0.001874,-0.001808,0.010517,0.005106,-0.006488,-0.006739,-0.000863,-0.015017,-0.012057,0.000549,-0.006825,-0.006421,-0.008743,0.000126,-0.011754,0.017352,-0.014569,0.000048,-0.008612
SeniorCitizen,-0.001874,1.000000,0.016479,-0.211185,0.016567,0.008576,0.146185,-0.032310,-0.128221,-0.013632,-0.021398,-0.151268,0.030776,0.047266,-0.142554,0.156530,-0.038551,0.220173,0.102411,0.150889
Partner,-0.001808,0.016479,1.000000,0.452676,0.379697,0.017706,0.142410,0.000891,0.150828,0.153130,0.166330,0.126733,0.137341,0.129574,0.294806,-0.014877,-0.154798,0.096848,0.319072,-0.150448
Dependents,0.010517,-0.211185,0.452676,1.000000,0.159712,-0.001762,-0.024991,0.044590,0.152166,0.091015,0.080537,0.133524,0.046885,0.021321,0.243187,-0.111377,-0.040292,-0.113890,0.064653,-0.164221
tenure,0.005106,0.016567,0.379697,0.159712,1.000000,0.008448,0.343032,-0.030359,0.325468,0.370876,0.371105,0.322942,0.289373,0.296866,0.671607,0.006152,-0.370436,0.247900,0.825880,-0.352229
PhoneService,-0.006488,0.008576,0.017706,-0.001762,0.008448,1.000000,-0.020538,0.387436,-0.015198,0.024105,0.003727,-0.019158,0.055353,0.043870,0.002247,0.016505,-0.004184,0.247398,0.113008,0.011942
MultipleLines,-0.006739,0.146185,0.142410,-0.024991,0.343032,-0.020538,1.000000,-0.109216,0.007141,0.117327,0.122318,0.011466,0.175059,0.180957,0.110842,0.165146,-0.176793,0.433576,0.453202,0.038037
InternetService,-0.000863,-0.032310,0.000891,0.044590,-0.030359,0.387436,-0.109216,1.000000,-0.028416,0.036138,0.044944,-0.026047,0.107417,0.098350,0.099721,-0.138625,0.086140,-0.323260,-0.175691,-0.047291
OnlineSecurity,-0.015017,-0.128221,0.150828,0.152166,0.325468,-0.015198,0.007141,-0.028416,1.000000,0.185126,0.175985,0.285028,0.044669,0.055954,0.374416,-0.157641,-0.096726,-0.053878,0.254473,-0.289309
OnlineBackup,-0.012057,-0.013632,0.153130,0.091015,0.370876,0.024105,0.117327,0.036138,0.185126,1.000000,0.187757,0.195748,0.147186,0.136722,0.280980,-0.013370,-0.124847,0.119777,0.375556,-0.195525


In [75]:
features=['tenure','OnlineSecurity','TechSupport', 'Contract','PaperlessBilling','OnlineBackup','Churn']

In [76]:
newdataSet= newdata[features]
newdataSet

,tenure,OnlineSecurity,TechSupport,Contract,PaperlessBilling,OnlineBackup,Churn
0,1,0,0,0,1,2,0
1,34,2,0,1,0,0,0
2,2,2,0,0,1,2,1
3,45,2,2,1,0,0,0
4,2,0,0,0,1,0,1
...,...,...,...,...,...,...,...
7038,24,2,2,1,1,0,0
7039,72,0,0,1,1,2,0
7040,11,2,0,0,1,0,0
7041,4,0,0,0,1,0,1


In [77]:
newdataSet.shape

(7043, 7)

In [78]:
X=newdataSet[['tenure','OnlineSecurity','TechSupport', 'Contract','PaperlessBilling','OnlineBackup']]
Y=newdataSet['Churn']

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [80]:
predictingModel=xgb.XGBClassifier(n_estimators=200,max_depth=5)
predictingModel.fit(X_train.values,y_train.values)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [81]:
y_prediction=predictingModel.predict(X_test.values)

In [83]:
accuracy_score(y_test.values, y_prediction)

0.751596877217885

In [84]:
pickle.dump(predictingModel,open('models.pkl','wb'))